# Spectral Similarity Tool
In this notebook we will use Descartes Labs `Workflows` to **explore the spectral similarity of minerals** and other materials in a remote region of Argentina.  You can draw your own AOI in the map created below, and we'll find nearby areas with a similar spectral signature in infrared imagery (SWIR and NIR) from Sentinel-2.

You can run the following cells using `Shift-Enter`.

## Import packages

In [ ]:
# keep logging quiet
import logging
logging.getLogger().setLevel(logging.INFO)
logging.captureWarnings(True)

In [ ]:
# import packages
import descarteslabs as dl
from descarteslabs import workflows as wf
import ipyleaflet

## Define the Sentinel-2 imagery that we'll be using
We will visualize two layers in the map below: a _SWIR/NIR_ layer and a _RGB_ layer.  Separately we will use _SWIR/NIR/Red_ bands to define the spectral similarity.

In [ ]:
# Define a Workflows ImageCollection using Sentinel-2 imagery.
s2 = (
    wf.ImageCollection.from_id("sentinel-2:L1C",
                                start_datetime='2019-08-25',
                                end_datetime='2019-09-05')
    .filter(lambda img: img.properties['cloud_fraction'] < 0.5)
    .median(axis='images')
)

# Visualize the Red-Green-Blue bands.
rgb = s2.pick_bands("red green blue")
rgb.visualize("RGB", scales=[[0,0.4],[0,0.4],[0,0.4]])

# Visualize the SWIR and NIR bands.
swir = s2.pick_bands("swir2 swir1 nir")
swir.visualize("SWIR/NIR", scales=[[0,0.6],[0,0.6],[0,0.6]])

# Define SPECTRAL SIMILARITY.
# First, here are the bands we'll use to calculate spectral similarity.
spectral_similarity_bands = ['swir2','swir1','nir','red']

# Next, define normalized band differences between these bands to be used for the actual spectral distance.
derived_bands = [wf.normalized_difference(*s2.unpack_bands("{} {}".format(b1, b2))).rename_bands("{}".format(ib)) for (ib, (b1, b2)) in enumerate(zip(spectral_similarity_bands[1:], spectral_similarity_bands[:-1]))]
band_stack = derived_bands[0]
for db in derived_bands[1:]:
    band_stack = band_stack.concat_bands(db)

## Define custom ipywidget to calculate and display spectral similarity.
This callback function calculates the mean spectrum of the AOI, calculates the spectral "distance" to that spectrum for all pixels in the image, and displays the spectral similarity (inverse of spectral distance) as a new layer in the map below.

In [ ]:
draw_control = ipyleaflet.DrawControl(
    edit=False,
    remove=False,
    circlemarker={},
    polyline={},
    polygon={"shapeOptions": {
        "fillColor": "#d534eb",
        "color": "#d534eb",
        "fillOpacity": 0.2
    }},
    rectangle={"shapeOptions": {
        "fillColor": "#d534eb",
        "color": "#d534eb",
        "fillOpacity": 0.2
    }}
)
wf.map.add_control(draw_control)

@wf.map.output_log.capture()
def callback(*args, **kwargs):
    last_draw = draw_control.last_draw
    if last_draw['geometry']['type'] == 'Point':
        auger_context = wf.GeoContext.from_dltile_key(
                    dl.raster.dltile_from_latlon(
                        draw_control.last_draw['geometry']['coordinates'][1],
                        draw_control.last_draw['geometry']['coordinates'][0],
                        156543.00/2**(max(wf.map.map.zoom, 0)), 2, 0).properties.key)

    elif last_draw['geometry']['type'] == 'Polygon': 
        auger_context = wf.GeoContext(
            geometry=last_draw['geometry'],
            resolution=156543.00/2**(max(wf.map.map.zoom, 0)),
            crs='EPSG:3857',
            bounds_crs='EPSG:4326',
        )
    reference = band_stack.median(axis='pixels').compute(auger_context)
    
    distance = None
    for ib, db in enumerate(derived_bands):
        b = "{}".format(ib)
        band_data = band_stack.pick_bands(b)
        if distance is None:
            distance = ((band_data - reference[b])*(band_data - reference[b]))
        else:
            distance += ((band_data - reference[b])*(band_data - reference[b]))
    inverse_distance = (len(spectral_similarity_bands)/wf.sqrt(distance)).rename_bands("distance")
    ly = inverse_distance.visualize("Spectral Similarity", scales=[25, 35], colormap='magma')
    ly.opacity = 0.6
draw_control.on_draw(callback)

## Initialize Workflows map and define Area of Interest (AOI)

In [ ]:
wf.map.center = [-37.2481, -69.8514] # geology and snow in South America
wf.map.zoom = 12

## Finally, display and interact with the map.
This map shows a remote area of western Argentina where some surface minerals and snow show distinctive signatures in the SWIR/NIR part of the spectrum.  You can use the polygon tool on the left side of the map to define your own AOI.  This will trigger the backend computation of spectral similarity to all pixels in this map (or anywhere in the world that you pan to).  Think of this as a quick and simple way to "show me more pixels that look like this".

In [ ]:
wf.map